In [2]:
!pip install konlpy python-mecab-ko wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.6/575.6 KB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 16.1 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=cd7a2a1977e86ad02e66fe3ff21ad8ba680ce30302b1dd827eb136c6a779b8c5
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud
from collections import Counter
import wget, os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [43]:
data_path = '/content/drive/MyDrive/에이블스쿨/실습파일/2023.04.03_미니프로젝트4차_실습자료/train.csv'
data = pd.read_csv(data_path)

In [44]:
import re

def remove_i(text):
    result = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', text)
    return result

In [45]:
data['text'] = data['text'].apply(remove_i)

In [46]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

In [47]:
data['label'] = data['label'].replace(label_dict)

In [48]:
def nouns_mecab(text):
    mecab = MeCab()
    nouns = mecab.nouns(text)
    return nouns

In [49]:
data['text_k_nouns'] = data['text'].apply(nouns_mecab)

In [50]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
import re

def nouns_e_nltk(text):
    text = text.lower()
    word_tokens = nltk.word_tokenize(text)
    tokens_pos = nltk.pos_tag(word_tokens)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos:
            NN_words.append(word)
  
    wlem = nltk.WordNetLemmatizer()
    lemmatized_words = []
    for word in NN_words:
        new_word = wlem.lemmatize(word)
        lemmatized_words.append(new_word)

    stopwords_list = stopwords.words('english') #nltk에서 제공하는 불용어사전 이용
    unique_NN_words = set(lemmatized_words)
    final_NN_words = lemmatized_words
    for word in unique_NN_words:
        if word in stopwords_list:
            while word in final_NN_words: 
                final_NN_words.remove(word)
    final_NN_words = list(map(str, re.sub('[^a-zA-Z0-9]',' ',str(final_NN_words)).strip().split()))
    return final_NN_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [51]:
data['text_e_nouns'] = data['text'].apply(nouns_e_nltk)

In [52]:
data['text_nouns'] = data['text_k_nouns']+data ['text_e_nouns']

In [53]:
target = 'label'
x = data.drop(target,axis=1, inplace=False)
y = data.loc[:,target]

In [69]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2023, stratify=y)

In [70]:
filename = '/content/drive/MyDrive/에이블스쿨/실습파일/2023.04.03_미니프로젝트4차_실습자료/불용어.txt'
f = open(filename, 'r', encoding='UTF8')
stop = f.read()

In [71]:
stopwords = list(map(str, stop.split()))

In [72]:
Tfidf_vect = TfidfVectorizer(max_features=5000, stop_words = stopwords)

In [73]:
x_train['text_nouns'] = x_train['text_nouns'].astype(str)
x_test['text_nouns'] = x_test['text_nouns'].astype(str)

In [74]:
Tfidf_vect.fit(x_train['text_nouns'])

TfidfVectorizer(max_features=5000,
                stop_words=['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희',
                            '따라', '의해', '을', '를', '에', '의', '가', '으로', '로',
                            '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면',
                            '예를', '들면', '예를', '들자면', '저', ...])

In [75]:
Train_X_Tfidf = Tfidf_vect.transform(x_train['text_nouns'])
Test_X_Tfidf = Tfidf_vect.transform(x_test['text_nouns'])

In [76]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(Train_X_Tfidf, y_train)

LogisticRegression(C=20, max_iter=1000)

In [77]:
predictions_lr = lr.predict(Test_X_Tfidf)
print("lr Accuracy Score -> ",accuracy_score(predictions_lr, y_test)*100)

lr Accuracy Score ->  83.82749326145552


In [78]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=5, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)

SVC(degree=5, gamma='auto', kernel='linear')

In [79]:
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  82.74932614555256


In [80]:
model_RF = RandomForestClassifier(random_state=1)

In [81]:
# 파라미터 선언
  # max_depth: range(1, 51)
param = {'max_depth': range(1,51)}

# Grid Search 선언
  # cv=5
  # scoring='accuracy'
model = GridSearchCV(model_RF,
                     param,
                     cv=5,
                     scoring='accuracy')

In [82]:
model.fit(Train_X_Tfidf,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
             param_grid={'max_depth': range(1, 51)}, scoring='accuracy')

In [83]:
predictions_RFC = model.predict(Test_X_Tfidf)

In [84]:
print("RFC Accuracy Score -> ",accuracy_score(predictions_RFC, y_test)*100)

RFC Accuracy Score ->  76.41509433962264


In [85]:
Train_X_Tfidf.shape

(2964, 5000)